<a href="https://colab.research.google.com/github/ethankyle360/ML4SupplyChain/blob/main/Section_16_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this assignment, we will work on the UK and Ireland dataset and create a re-ordering policy for every SKU's assuming we have a common warehouse for both UK customers and Ireland.

Instructions:
1.   Make a new script.
2.   Filter the last 90 days of the UK and Ireland.
3.   Transform the data to make ABC analysis and set the service level. Calculate for the ABC.
4.   Calculate the average daily demand and the standard deviation per SKU. Do the ABC analysis and join them with the expected sales and standard deviation. Set service level for
A_A, B_A, C_A to 80%, A_C, A_B, B_B to 70%, C_C, C_B, B_C to 60%

5. Calculate the re-order point per each SKU's. Let's assume that the lead time is 21 days for all the SKU's. Also, assume that the standard deviation of the lead time is 2 days for all SKU's.

In [ ]:
!pip install inventorize1

In [2]:
import pandas as pd
import inventorize1 as inv
import datetime
import numpy as np
retail = pd.read_csv('Uk_Ireland.csv')

In [3]:
retail = retail.drop_duplicates()
retail = retail.dropna(axis=0)

In [4]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 598177 entries, 0 to 790077
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      598177 non-null  float64
 1   StockCode    598177 non-null  object 
 2   Description  598177 non-null  object 
 3   Quantity     598177 non-null  float64
 4   InvoiceDate  598177 non-null  object 
 5   Price        598177 non-null  float64
 6   Customer.ID  598177 non-null  float64
 7   Country      598177 non-null  object 
 8   date         598177 non-null  object 
 9   revenue      598177 non-null  float64
 10  year         598177 non-null  float64
 11  month        598177 non-null  object 
dtypes: float64(6), object(6)
memory usage: 59.3+ MB


In [5]:
retail.InvoiceDate

,InvoiceDate
0,2009-12-01T07:45:00Z
1,2009-12-01T07:45:00Z
2,2009-12-01T07:45:00Z
3,2009-12-01T07:45:00Z
4,2009-12-01T07:45:00Z
...,...
790073,2011-10-02T12:45:00Z
790074,2011-10-02T12:45:00Z
790075,2011-10-02T12:45:00Z
790076,2011-10-02T12:45:00Z


In [6]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

In [7]:
retail['date'] = retail.InvoiceDate.dt.strftime('%Y-%m-%d')

In [8]:
retail['date'] = pd.to_datetime(retail['date'])

In [9]:
max_data = max(retail.date)

In [10]:
last_three = retail[retail.date > '2011-09-09']

In [11]:
last_three.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer.ID', 'Country', 'date', 'revenue', 'year', 'month'],
      dtype='object')

In [14]:
last_three['revenue'] = last_three['Quantity'] * last_three['Price']

<ipython-input-14-a0e20c47a427>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_three['revenue'] = last_three['Quantity'] * last_three['Price']


In [16]:
a = last_three.groupby(['date', 'Description']).agg(total_daily = ('Quantity', np.sum),
                                                total_revenue = ('revenue', np.sum)).reset_index()

<ipython-input-16-e943339f7280>:1: FutureWarning: The provided callable <function sum at 0x78e125316e60> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  a = last_three.groupby(['date', 'Description']).agg(total_daily = ('Quantity', np.sum),
<ipython-input-16-e943339f7280>:1: FutureWarning: The provided callable <function sum at 0x78e125316e60> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  a = last_three.groupby(['date', 'Description']).agg(total_daily = ('Quantity', np.sum),


In [18]:
grouped = a.groupby('Description').agg(average = ('total_daily', np.mean),
                                       sd = ('total_daily', 'std'),
                                       total_sales = ('total_daily', np.sum),
                                       total_revenue = ('total_revenue', np.sum)).reset_index()

<ipython-input-18-814c3bd2b4d9>:1: FutureWarning: The provided callable <function mean at 0x78e125317d90> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped = a.groupby('Description').agg(average = ('total_daily', np.mean),
<ipython-input-18-814c3bd2b4d9>:1: FutureWarning: The provided callable <function sum at 0x78e125316e60> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  grouped = a.groupby('Description').agg(average = ('total_daily', np.mean),
